In [ ]:
from dspy.datasets import HotPotQA

hotpot_dataset = HotPotQA(train_seed = 1, eval_seed = 2023, dev_size = 20, keep_details = True)
example_data = hotpot_dataset.dev[0]

context_text = "\n\n".join("\n".join(focused_context) for focused_context in example_data.context["sentences"])

In [ ]:
from cognee.modules.cognify.extract_categories import ExtractCategories

extract_categories = ExtractCategories()

categories = extract_categories(text = context_text)
print(categories)


In [ ]:
from cognee.modules.cognify.extract_cognitive_layers import ExtractCognitiveLayers
from cognee.shared.data_models import TextContent

extract_cognitive_layers = ExtractCognitiveLayers()

category = categories[0].subclass[0].value

cognitive_layers = extract_cognitive_layers(text = context_text, category = category)

print(cognitive_layers)


In [ ]:
import dspy
from uuid import uuid4
from cognee.infrastructure.databases.graph.get_graph_client import get_graph_client, GraphDBType
from cognee.modules.cognify.generate_knowledge_graph import GenerateKnowledgeGraph
from cognee.shared.data_models import Node, Edge

dspy.configure(trace=[])

generate_knowledge_graph = GenerateKnowledgeGraph().activate_assertions()

graph_file_name = str(uuid4()) + ".pkl"

graph_client = get_graph_client(GraphDBType.NETWORKX, graph_file_name)

graph = generate_knowledge_graph(layer = 'Transportation Infrastructure', text = context_text)

root_node_per_category = {}

for node in graph.nodes:
    if node.entity_type is not None and node.entity_name is not None:
        graph_client.add_node(node.id, entity_name = node.entity_name, entity_type = node.entity_type)

        if node.entity_type not in root_node_per_category:
            root_node = Node(
                id = node.entity_type + " root",
                entity_name = node.entity_type,
                entity_type = node.entity_type + " root"
            )
            root_node_per_category[node.entity_type] = root_node
            graph_client.add_node(
                id = root_node.id,
                entity_name = root_node.entity_name,
                entity_type = root_node.entity_type
            )

        graph_client.add_edge(
            node.id,
            root_node_per_category[node.entity_type].id,
            relationship_name = "is"
        )

for edge in graph.edges:
    if edge.source_node_id is not None and edge.target_node_id is not None and edge.relationship_name is not None:
        graph_client.add_edge(edge.source_node_id, edge.target_node_id, relationship_name = edge.relationship_name)



In [3]:
import networkx as nx
import pandas as pd
import graphistry
from cognee.config import Config
from cognee.utils import render_graph
from cognee.infrastructure.databases.graph.get_graph_client import get_graph_client, GraphDBType

config = Config()
config.load()

graphistry.register(
    api = 3,
    username = config.graphistry_username,
    password = config.graphistry_password
)

graph_client = get_graph_client(GraphDBType.NETWORKX, "32652370-04d9-418e-916d-3086aa41685c.pkl")
graph = graph_client.graph

edges = nx.to_pandas_edgelist(graph)

nodes_data = [{
    "id": node_id,
    "entity_name": node["entity_name"],
    "entity_type": node["entity_type"]
} for (node_id, node) in graph.nodes(data = True)]

nodes = pd.DataFrame(nodes_data)

plotter = graphistry.edges(edges, source = "source", destination = "target").nodes(nodes, "id")

plotter.bind(edge_title = "relationship_name", edge_label = "relationship_name", point_title = "entity_name", point_label = "entity_name")
url = plotter.plot(render = False, as_files = True)
print(f"Graph is visualized at: {url}")

Graph is visualized at: https://hub.graphistry.com/graph/graph.html?dataset=842a911115124473bbf23f2769dc3e96&type=arrow&viztoken=65c1d750-91fa-4e42-8696-6e8e000c34ae&usertag=993172cb-pygraphistry-0.33.5&splashAfter=1712859766&info=true
